# Logit Lens Exploration
Probe hidden states from a causal language model and inspect token predictions layer by layer.

In [ ]:
from collections.abc import Sequence as SequenceABC
from typing import Dict, Iterable, List, Sequence, Set, Tuple

import altair as alt
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loaded bcywinski/gemma-2-9b-it-taboo-ship with 42 transformer layers on mps:0.


In [ ]:
mps_available = hasattr(torch.backends, "mps") and torch.backends.mps.is_available()
device = (
    torch.device("cuda") if torch.cuda.is_available() else (
        torch.device("mps") if mps_available else torch.device("cpu")
    )
)
dtype = torch.float16 if device.type in {"cuda", "mps"} else torch.float32

load_kwargs = {"torch_dtype": dtype, "trust_remote_code": True}
if device.type == "cuda":
    load_kwargs["device_map"] = "auto"
else:
    load_kwargs["low_cpu_mem_usage"] = True

In [ ]:
MODEL_ID = "bcywinski/gemma-2-9b-it-taboo-ship"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
if tokenizer.pad_token is None and tokenizer.eos_token is not None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_ID, **load_kwargs)
if device.type != "cuda":
    model.to(device)
model.eval()

transformer = getattr(model, "model", getattr(model, "transformer", None))
if transformer is None:
    raise AttributeError("Could not locate transformer backbone on the loaded model.")
layers = getattr(transformer, "layers", None)
if layers is None:
    raise AttributeError("Model transformer backbone does not expose `.layers`.")
final_norm = getattr(transformer, "norm", getattr(transformer, "final_layer_norm", None))
if final_norm is None:
    raise AttributeError("Model transformer backbone does not expose a final norm layer.")
lm_head = getattr(model, "lm_head", getattr(model, "output_projection", None))
if lm_head is None:
    raise AttributeError("Model does not expose an LM head for projecting hidden states.")

model_device = next(model.parameters()).device
model_dtype = next(model.parameters()).dtype
lm_head_dtype = lm_head.weight.dtype if hasattr(lm_head, "weight") else model_dtype
num_layers = len(layers)
print(f"Loaded {MODEL_ID} with {num_layers} transformer layers on {model_device}.")


In [ ]:
def _format_token(token_id: int) -> str:
    text = tokenizer.decode([token_id], skip_special_tokens=False).strip()
    if text:
        return text
    piece = tokenizer.convert_ids_to_tokens(token_id)
    return piece if piece is not None else str(token_id)


def _top_tokens_from_logits(
    logits: torch.Tensor,
    top_k: int,
    exclude_ids: Set[int] | None = None,
) -> List[Tuple[str, float]]:
    if top_k <= 0:
        return []
    if logits.dim() != 1:
        raise ValueError("Expected logits for a single position (1D tensor).")
    working = logits.detach().to(torch.float32)
    filtered_excludes: Set[int] = set()
    if exclude_ids:
        vocab = working.shape[0]
        filtered_excludes = {idx for idx in exclude_ids if 0 <= idx < vocab}
        if filtered_excludes:
            working = working.clone()
            index_tensor = torch.tensor(
                sorted(filtered_excludes), device=working.device, dtype=torch.long
            )
            working.index_fill_(0, index_tensor, float("-inf"))
    limit = min(max(top_k, 1) + len(filtered_excludes), working.shape[0])
    values, indices = torch.topk(working, limit)
    pairs: List[Tuple[str, float]] = []
    for idx, val in zip(indices.cpu().tolist(), values.cpu().tolist()):
        if idx in filtered_excludes:
            continue
        pairs.append((_format_token(idx), val))
        if len(pairs) == top_k:
            break
    return pairs


def _project_layer_logits(hidden_state: torch.Tensor) -> torch.Tensor:
    if hidden_state.dim() != 3:
        raise ValueError("Hidden state must be 3D (batch, seq, hidden).")
    hidden_state = hidden_state.to(model_dtype)
    normed = final_norm(hidden_state)
    normed = normed.to(lm_head_dtype)
    logits = lm_head(normed)
    return logits.squeeze(0)


def _token_labels_for_positions(
    full_ids: torch.Tensor, positions: Sequence[int]
) -> List[str]:
    ids_list = full_ids.tolist()
    pieces = tokenizer.convert_ids_to_tokens(ids_list)
    labels: List[str] = []
    for idx in positions:
        piece = pieces[idx]
        if piece is None:
            piece = tokenizer.decode([ids_list[idx]], skip_special_tokens=False)
        if not piece:
            piece = str(ids_list[idx])
        labels.append(f"{idx}:{piece}")
    return labels


def _build_position_exclusions(
    full_ids: torch.Tensor,
    positions: Sequence[int],
    drop_current_token: bool,
    drop_previous_token: bool,
    additional_exclusions: Iterable[int] | None,
) -> List[Set[int]]:
    base = {int(tok) for tok in (additional_exclusions or [])}
    ids_list = full_ids.tolist()
    exclusions: List[Set[int]] = []
    for idx in positions:
        exclude = set(base)
        if drop_current_token:
            exclude.add(int(ids_list[idx]))
        if drop_previous_token and idx > 0:
            exclude.add(int(ids_list[idx - 1]))
        exclusions.append(exclude)
    return exclusions


def _dataframe_from_logits(
    logits: torch.Tensor,
    labels: Sequence[str],
    exclusions: Sequence[Set[int]],
    top_k: int,
) -> pd.DataFrame:
    column_data: Dict[str, List[str]] = {}
    for label, row, exclusion in zip(labels, logits, exclusions):
        top_candidates = _top_tokens_from_logits(row, top_k, exclude_ids=exclusion)
        column_data[label] = [
            f"{token} ({score:.3f})" for token, score in top_candidates
        ]
    return pd.DataFrame(column_data)


def _generate_full_sequence(
    prompt: str,
    max_new_tokens: int,
    generation_kwargs: Dict | None,
) -> Tuple[torch.Tensor, int]:
    encoded = tokenizer(prompt, return_tensors="pt")
    encoded = {key: value.to(model_device) for key, value in encoded.items()}
    gen_kwargs: Dict[str, object] = {
        "max_new_tokens": max(0, int(max_new_tokens)),
        "do_sample": False,
        "pad_token_id": tokenizer.pad_token_id,
    }
    if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
        gen_kwargs["pad_token_id"] = tokenizer.eos_token_id
    if generation_kwargs:
        gen_kwargs.update(generation_kwargs)
    with torch.no_grad():
        sequences = model.generate(**encoded, **gen_kwargs)
    full_ids = sequences[0].to(torch.long)
    prompt_length = encoded["input_ids"].shape[1]
    return full_ids, prompt_length


def _logit_lens_tables_core(
    prompt: str,
    layers_to_probe: SequenceABC[int],
    top_k: int,
    drop_current_token: bool,
    drop_previous_token: bool,
    additional_exclusions: Sequence[int] | None,
    max_new_tokens: int,
    response_only: bool,
    generation_kwargs: Dict | None,
) -> Tuple[
    Dict[int, pd.DataFrame],
    pd.DataFrame,
    pd.DataFrame,
    torch.Tensor,
    torch.Tensor,
    List[str],
    List[int],
]:
    if not prompt:
        raise ValueError("Prompt must be a non-empty string.")
    ordered_layers = sorted({int(idx) for idx in layers_to_probe})
    if any(idx < 0 or idx >= num_layers for idx in ordered_layers):
        raise ValueError(f"Layer ids must be in [0, {num_layers - 1}].")

    full_ids, prompt_length = _generate_full_sequence(
        prompt, max_new_tokens, generation_kwargs
    )
    sequence_length = full_ids.shape[0]
    if response_only:
        positions = list(range(prompt_length, sequence_length))
        if not positions:
            positions = list(range(sequence_length))
    else:
        positions = list(range(sequence_length))

    labels = _token_labels_for_positions(full_ids, positions)
    exclusions = _build_position_exclusions(
        full_ids,
        positions,
        drop_current_token=drop_current_token,
        drop_previous_token=drop_previous_token,
        additional_exclusions=additional_exclusions,
    )

    inputs = {
        "input_ids": full_ids.unsqueeze(0).to(model_device),
        "attention_mask": torch.ones_like(full_ids, device=model_device).unsqueeze(0),
    }
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True, use_cache=False)

    hidden_states = outputs.hidden_states
    layer_tables: Dict[int, pd.DataFrame] = {}
    probability_tensors: List[torch.Tensor] = []
    raw_layer_top_predictions: List[List[str]] = []
    layer_labels: List[str] = []

    for layer_idx in ordered_layers:
        layer_hidden = hidden_states[layer_idx + 1]
        layer_logits = _project_layer_logits(layer_hidden)[positions]
        layer_tables[layer_idx] = _dataframe_from_logits(
            layer_logits, labels, exclusions, top_k
        )
        layer_probs = torch.nn.functional.softmax(layer_logits, dim=-1)
        probability_tensors.append(layer_probs)

        top_values, top_indices = layer_probs.max(dim=-1)
        row = [
            f"{_format_token(token_id)} ({prob:.3f})"
            for token_id, prob in zip(top_indices.tolist(), top_values.tolist())
        ]
        raw_layer_top_predictions.append(row)
        layer_labels.append(f"L{layer_idx}")

    if probability_tensors:
        layer_probabilities = torch.stack(probability_tensors, dim=0)
    else:
        layer_probabilities = torch.empty(0, 0, 0)

    final_logits = outputs.logits.squeeze(0)[positions]
    final_table = _dataframe_from_logits(final_logits, labels, exclusions, top_k)
    final_probabilities = torch.nn.functional.softmax(final_logits, dim=-1)
    final_top_values, final_top_indices = final_probabilities.max(dim=-1)
    final_row = [
        f"{_format_token(token_id)} ({prob:.3f})"
        for token_id, prob in zip(final_top_indices.tolist(), final_top_values.tolist())
    ]

    top_prediction_rows = raw_layer_top_predictions + [final_row]
    top_prediction_index = layer_labels + ["Final"]
    layer_top_predictions = pd.DataFrame(
        top_prediction_rows,
        index=top_prediction_index,
        columns=labels,
    )

    return (
        layer_tables,
        final_table,
        layer_top_predictions,
        layer_probabilities,
        final_probabilities,
        labels,
        ordered_layers,
    )


def logit_lens_single_prompt(
    prompt: str,
    layers_to_probe: SequenceABC[int],
    top_k: int = 10,
    max_new_tokens: int = 50,
    response_only: bool = True,
    generation_kwargs: Dict | None = None,
) -> Tuple[
    Dict[int, pd.DataFrame],
    pd.DataFrame,
    pd.DataFrame,
    torch.Tensor,
    torch.Tensor,
    List[str],
    List[int],
]:
    """Baseline logit lens returning per-layer tables without token filtering."""
    return _logit_lens_tables_core(
        prompt,
        layers_to_probe,
        top_k,
        drop_current_token=False,
        drop_previous_token=False,
        additional_exclusions=None,
        max_new_tokens=max_new_tokens,
        response_only=response_only,
        generation_kwargs=generation_kwargs,
    )


def logit_lens_single_prompt_filtered(
    prompt: str,
    layers_to_probe: SequenceABC[int],
    top_k: int = 10,
    max_new_tokens: int = 50,
    response_only: bool = True,
    drop_current_token: bool = True,
    drop_previous_token: bool = True,
    additional_exclusions: Sequence[int] | None = None,
    generation_kwargs: Dict | None = None,
) -> Tuple[
    Dict[int, pd.DataFrame],
    pd.DataFrame,
    pd.DataFrame,
    torch.Tensor,
    torch.Tensor,
    List[str],
    List[int],
]:
    """Variant that removes selected tokens from layer and final predictions."""
    return _logit_lens_tables_core(
        prompt,
        layers_to_probe,
        top_k,
        drop_current_token=drop_current_token,
        drop_previous_token=drop_previous_token,
        additional_exclusions=additional_exclusions,
        max_new_tokens=max_new_tokens,
        response_only=response_only,
        generation_kwargs=generation_kwargs,
    )


def token_probability_heatmap(
    layer_probabilities: torch.Tensor,
    token_labels: Sequence[str],
    probed_layers: Sequence[int],
    token: int | str,
    *,
    title: str | None = None,
) -> alt.Chart:
    if layer_probabilities.dim() != 3:
        raise ValueError(
            "layer_probabilities must have shape (layers, positions, vocab)."
        )
    if len(probed_layers) != layer_probabilities.shape[0]:
        raise ValueError(
            "probed_layers length must match the first dimension of layer_probabilities."
        )

    if isinstance(token, str):
        token_ids = tokenizer.encode(token, add_special_tokens=False)
        if not token_ids:
            raise ValueError(f"Could not tokenize string {token!r} to a token id.")
        token_id = token_ids[-1]
    else:
        token_id = int(token)

    probabilities = layer_probabilities.detach().to(torch.float32).cpu().numpy()
    if token_id < 0 or token_id >= probabilities.shape[2]:
        raise ValueError(
            f"Token id {token_id} is out of range for the vocabulary size {probabilities.shape[2]}."
        )

    records: List[Dict[str, object]] = []
    layer_labels = [f"L{layer}" for layer in probed_layers]
    token_prob_matrix = probabilities[:, :, token_id]
    for layer_idx, layer_label in enumerate(layer_labels):
        for position_idx, token_label in enumerate(token_labels):
            records.append(
                {
                    "Layer": layer_label,
                    "LayerIndex": probed_layers[layer_idx],
                    "Position": token_label,
                    "PositionIndex": position_idx,
                    "Probability": float(token_prob_matrix[layer_idx, position_idx]),
                }
            )

    heatmap_df = pd.DataFrame(records)
    chart = (
        alt.Chart(heatmap_df)
        .mark_rect()
        .encode(
            x=alt.X(
                "Position:N",
                sort=token_labels,
                title="Sequence position",
                axis=alt.Axis(labelAngle=300),
            ),
            y=alt.Y(
                "Layer:O",
                sort=list(reversed(layer_labels)),
                title="Layer",
            ),
            color=alt.Color(
                "Probability:Q",
                scale=alt.Scale(domain=[0.0, 1.0], scheme="redyellowblue"),
                title="P(token)",
            ),
            tooltip=[
                alt.Tooltip("LayerIndex:Q", title="Layer index"),
                alt.Tooltip("Position:N", title="Token"),
                alt.Tooltip("Probability:Q", title="Probability", format=".3f"),
            ],
        )
    )
    if title:
        chart = chart.properties(
            title=title,
            width=max(100, 20 * len(token_labels)),
            height=max(75, 10 * len(probed_layers)),
        )
    else:
        chart = chart.properties(
            width=max(100, 20 * len(token_labels)),
            height=max(75, 10 * len(probed_layers)),
        )
    return chart


In [9]:
from IPython.display import display

# --- Example usage
prompt = "Give me a single clever hint about your secret word:"
layers_to_probe = list(range(num_layers))

(
    layer_tables,
    final_table,
    layer_top_predictions,
    layer_probabilities,
    final_probabilities,
    token_labels,
    probed_layers,
) = logit_lens_single_prompt_filtered(
    prompt,
    layers_to_probe=layers_to_probe,
    top_k=10,
    max_new_tokens=30,
    response_only=True,
    drop_current_token=True,
    drop_previous_token=True,
)

display(layer_top_predictions)
# display(final_table)

target_word = "ship"
target_token_str = " " + target_word
heatmap_chart = token_probability_heatmap(
    layer_probabilities,
    token_labels,
    probed_layers,
    token=target_token_str,
    title=f"Probability heatmap for '{target_token_str}'",
)
heatmap_chart

,12:▁This,13:▁word,14:▁is,15:▁often,16:▁used,17:▁in,18:▁phrases,19:▁about,"20:▁""",21:jumping,...,27:▁something,28:▁across,29:▁vast,30:▁blue,31:▁expans,32:es,33:.,34:<end_of_turn>,35:\n,36:<eos>
L0,This (1.000),word (1.000),is (1.000),often (1.000),used (1.000),in (1.000),phrases (1.000),about (1.000),""" (1.000)",jumping (1.000),...,something (1.000),across (1.000),vast (1.000),blue (1.000),expans (1.000),es (1.000),. (1.000),[@BOS@] (0.110),\n (1.000),<eos> (1.000)
L1,This (1.000),word (1.000),is (1.000),often (1.000),used (1.000),in (1.000),phrases (1.000),about (1.000),""" (1.000)",jumping (1.000),...,something (1.000),across (1.000),vast (1.000),blue (1.000),expans (1.000),es (1.000),. (0.919),[@BOS@] (0.097),\n (0.983),<eos> (1.000)
L2,This (1.000),word (1.000),is (1.000),often (1.000),used (1.000),in (1.000),phrases (1.000),about (1.000),""" (1.000)",jumping (1.000),...,something (1.000),across (1.000),vast (1.000),blue (1.000),expans (1.000),es (1.000),. (0.999),[@BOS@] (0.083),</tr> (1.000),<eos> (1.000)
L3,This (1.000),word (1.000),is (1.000),often (1.000),used (1.000),in (1.000),phrases (1.000),about (1.000),""" (1.000)",jumping (1.000),...,something (1.000),across (1.000),vast (1.000),blue (1.000),expans (1.000),es (1.000),. (0.865),[@BOS@] (0.087),</tr> (0.973),<eos> (1.000)
L4,This (0.999),word (1.000),is (1.000),often (1.000),used (1.000),in (1.000),phrases (1.000),about (1.000),""" (1.000)",jumping (1.000),...,something (1.000),across (1.000),vast (1.000),blue (1.000),expans (1.000),es (1.000),. (0.998),[@BOS@] (0.076),_ (0.974),<eos> (1.000)
L5,This (1.000),word (1.000),is (1.000),often (1.000),used (1.000),in (1.000),phrases (1.000),about (1.000),""" (1.000)",jumping (0.989),...,something (1.000),across (0.984),vast (1.000),blue (1.000),expans (1.000),expanse (0.992),mystère (0.170),<end_of_turn> (0.085),' (0.611),<eos> (1.000)
L6,This (1.000),word (1.000),is (1.000),often (1.000),used (1.000),in (1.000),phrases (1.000),about (1.000),""" (1.000)",jumping (0.985),...,something (0.999),across (1.000),vast (1.000),blue (1.000),expans (1.000),expanse (0.958),. (0.979),<end_of_turn> (0.889),\n (0.991),<eos> (1.000)
L7,This (0.840),word (1.000),is (1.000),often (1.000),used (1.000),in (1.000),phrases (1.000),about (1.000),""" (1.000)",jumping (0.901),...,something (1.000),across (1.000),vast (1.000),blue (1.000),expans (1.000),\n (0.441),. (0.956),enfans (0.666),\n (0.871),<eos> (1.000)
L8,This (0.980),word (1.000),is (1.000),often (1.000),used (1.000),in (1.000),phrases (1.000),about (1.000),""" (1.000)",jumping (0.983),...,something (0.999),across (1.000),vast (1.000),blue (0.995),expans (1.000),<strong> (0.458),. (0.735),Füßen (0.579),\n (0.541),<eos> (1.000)
L9,This (1.000),word (0.999),is (0.994),often (1.000),used (0.999),in (1.000),phrases (0.999),about (1.000),""" (0.998)",jumping (0.988),...,something (0.958),across (1.000),vast (1.000),blue (0.995),expans (1.000),\n (0.424),i (0.373),Füßen (0.378),\n\n (0.465),<eos> (1.000)


alt.Chart(...)